<a href="https://colab.research.google.com/github/MohammadrezaPourreza/Tournament_selection_GA/blob/main/Tournament_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
from copy import deepcopy
import numpy as np
from numpy import linalg as LA
import pandas as pd
import random
from math import *

def evaluate_all(parents,function):
  output = []
  for parent in parents:
    output.append(evaluate(parent,function))
  return output

def evaluate(child,function):
  score = 0
  if function == "f1":
    for gene in child:
      score += gene**2
  if function == "f4":
    for gene in child:
      if score < abs(gene):
        score = abs(gene)
  if function == "f6":
     for gene in child:
       score += (gene+0.5)**2
  if function == "f9":
    for gene in child:
      score += (gene**2) - (10 * cos((2*pi)*gene)) + 10
  if function == "f8":
    for gene in child:
      score += (-1*gene)*sin(sqrt(abs(gene)))
  if function == "f3":
      for i in range(len(child)):
        sum2 = 0
        for j in range(i):
          sum2 += child[i]
        score += sum2**2
  if function == "f7":
    for i in range(len(child)):
      score += ((i+1)*(child[i])**4) + random.random()
  if function == "f5":
    for i in range (len(child)-1):
        score += (100 * (child[i+1] - (child[i])**2)**2) + (child[i]-1)**2
  if function == "Schwefel":
    for gene in child:
      if score < abs(gene):
        score = abs(gene)
    score = score**2
  if function == "Schwefel problem":
    for gene in child:
      score += gene*sin(sqrt(abs(gene)))
  return score


def select(P, q,function,meu):
  parents = []
  for i in range(meu):
    candidates = random.choices(list(P), k=q)
    scores = []
    for candidate in candidates:
      scores.append(evaluate(candidate,function))  
    candidates = [list(x) for _,x in sorted(zip(scores,candidates))]
    parents.append(np.array(deepcopy(candidates[0])))
  return parents

#------------------------------------------------

def crossover(P, meu):
    C = P.copy()
    for p in range(0, meu, 2):
      if LA.norm(P[p]) < LA.norm(P[p+1]):
        C[p] = P[p] + random.random()*np.subtract(P[p+1],P[p])
        C[p+1] = P[p] + random.random()*np.subtract(P[p+1],P[p])
      else:
        C[p] = P[p+1] + random.random()*np.subtract(P[p],P[p+1])
        C[p+1] = P[p]+1 + random.random()*np.subtract(P[p],P[p+1])
    return C

#------------------------------------------------

def mutate(children,sigma,n,function):
  ps = 0
  output = []
  for child in children:
    newchild = child + sigma*np.random.normal(size = n)
    if evaluate(newchild,function) < evaluate(child,function):
      ps += 1
    output.append(newchild)
  ps = ps / len(children)
  return output,ps

#------------------------------------------------

meu = 20 #Number of people
q = 5 #tournament selection parameter
runs = 10 #number of runs
sigma = 10 #Mutation
delta = 0.95#scaler
n = 10 #vector length
xmin = -30 #fuction minimum value
xmax = 30 #function maximum value
#implemented test functions = [f1,f4,f6,f9,f8,f3,f7,f5,Schwefel,Schwefel problem]
function = "Schwefel problem" #name of function
max_gen = 150 #Maximum generation
FB = np.zeros(runs) #Best Fitness
for run in range(runs):

    P = [[random.randint(floor(xmin),floor(xmax) ) for y in range(n)]for i in range(meu)]
    Fbest = np.zeros(max_gen)
    for t in range(max_gen):

        parents = select(P,q,function,meu)
    
        evaluation = evaluate_all(parents,function)
        Fbest[t] = np.min(evaluation)

        children = crossover(parents,meu )

        children,ps = mutate(children,sigma,n,function)

        if ps < 0.2:
          sigma = sigma * delta
        elif ps > 0.2:
          sigma = sigma / delta
        
        P = children.copy()
    FB[run] = np.min(Fbest)
print("average Fitness on " + str(runs) + " runs : " +  str(np.mean(FB)))

average Fitness on 10 runs : -215.90570411649952
